In [1]:
import nibabel as nib
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
from sklearn.cluster import KMeans

In [2]:
def load_nifti(file_path):
    # Load the NIfTI image
    nii_image = nib.load(file_path)

    # Access the image data as a Numpy array
    data_array = nii_image.get_fdata()

    return data_array, nii_image

def show_nifti(file_data, slice=20):
    plt.imshow(file_data[:, :, slice], cmap='gray')  # Display a single slice (change the slice index as needed)
    plt.title('NIfTI Image')
    plt.colorbar()
    plt.show()

In [3]:
# subject_id  = str(1)

# data_folder_path = os.path.join(os.getcwd(), f'../Lab 1/P2_data/{subject_id}/results_60_0.0001/')

In [4]:
# file_name_ss        = 'output_ss_60_0.0001.nii'
# file_name_merged    = 'merged_segmentation.nii'

# nifti_file_ss = os.path.join(data_folder_path, file_name_ss)
# nifti_file_merged = os.path.join(data_folder_path, file_name_merged)


In [5]:
# nifti_vol_ss, _ = load_nifti(nifti_file_ss)
# show_nifti(nifti_vol_ss) # ss

In [6]:
# nifti_vol_merged, nifti_vol_merged_nii = load_nifti(nifti_file_merged)
# show_nifti(nifti_vol_merged) # merged

In [7]:
# # Load your NIfTI volume (assuming you have a function called load_nifti that loads the data)
# # nifti_vol = load_nifti(nifti_file)

# # Reshape the volume to a 2D array for clustering
# # The shape (240, 240, 48) will be reshaped to (240 * 240 * 48, 1)
# data = selected_tissue.reshape(-1, 1)

# # Perform k-means clustering with k=3
# kmeans = KMeans(n_clusters=4, random_state=0, n_init='auto')
# kmeans.fit(data)

# # Get the cluster labels and reshape them back to the original volume shape
# # cluster_labels = kmeans.labels_.reshape(nifti_vol_ss.shape)


#### Class code notes

In [8]:
# Functions implementation
def remove_black_bg(subject_id=1, labels_gt_file='LabelsForTesting.nii', t1_file='T1.nii', t2_file='T2_FLAIR.nii'):
    '''Removes the black background from the skull stripped volume and returns a 1D array of the voxel intensities \
        of the pixels that falls in the True region of the mask, for GM, WM, and CSF.'''
    
    # selecting paths
    data_folder_path    = os.path.join(os.getcwd(), f'../Lab 1/P2_data/{subject_id}')

    # load the nifti files
    labels_nifti, _ = load_nifti(os.path.join(data_folder_path, labels_gt_file))
    t1_volume, _    = load_nifti(os.path.join(data_folder_path, t1_file))
    t2_volume, _    = load_nifti(os.path.join(data_folder_path, t2_file))

    # Selecting the tissues based on their labels
    labels_nifti_csf = labels_nifti == 1
    labels_nifti_gm  = labels_nifti == 2
    labels_nifti_wm  = labels_nifti == 3

    # Adding the tissues togather to make a single file for the 3 tissues
    tissue_labels    = labels_nifti_csf + labels_nifti_gm + labels_nifti_wm

    # selecting the voxel values from the skull stripped
    t1_selected_tissue = t1_volume[tissue_labels].flatten()
    t2_selected_tissue = t2_volume[tissue_labels].flatten()

    # put both tissues into the d-dimensional data vector
    tissue_data =  np.array([t1_selected_tissue, t2_selected_tissue]).T

    # The true mask labels count must equal to the number of voxels we segmented
    # np.count_nonzero(tissue_labels) returns the sum of pixel values that are True, the count should be equal to the number
    # of pixels in the selected tissue array
    assert np.count_nonzero(tissue_labels) == t1_selected_tissue.shape[0], 'Error while removing T1 black background.'
    assert np.count_nonzero(tissue_labels) == t2_selected_tissue.shape[0], 'Error while removing T2_FLAIR black background.'

    return tissue_data

def initialize_parameters(tissue_data, random=False, k=3):
    '''Initializes the model parameters and the weights at the beginning of EM algorithm. It returns the initialized parameters.
    '''
    cluster_means = []
    cluster_covariance_matrices = []
    alpha_k = (1/3)*np.ones((3,), dtype=int) # init with [1/3, 1/3, 1/3], sum = 1.0, better this way to include floating points

    if not random:

        for modality_idx in range(tissue_data.shape[1]):
            # initialization
            modality_covariance_metrices = []

            # Reshape the volume to a 2D array for clustering
            # The shape (240, 240, 48) will be reshaped to (240 * 240 * 48, 1)
            data = tissue_data[:, modality_idx].reshape(-1, 1)

            # Perform k-means clustering with k=3
            kmeans = KMeans(n_clusters=k, random_state=0, n_init='auto')
            kmeans.fit(data)

            # Get the cluster labels
            cluster_labels = kmeans.labels_
            centroids = kmeans.cluster_centers_

            # appending the centroids (means) directly into the cluster means
            cluster_means.append(centroids.ravel())

            # Calculate the means and covariance matrices for each cluster
            for cluster_id in range(k):  # Assuming k=3
                cluster_data = data[cluster_labels == cluster_id]
                
                # Calculate the covariance matrix for this cluster
                modality_covariance_metrices.append(
                    np.cov(cluster_data, rowvar=False))

            cluster_covariance_matrices.append(modality_covariance_metrices)

    else:
        # random initialization of the model parameters
        pass

    return cluster_means, cluster_covariance_matrices, alpha_k
    
    

In [9]:
# Pre-processing the tissues to remove the background and return them as a feature representation
tissue_data = remove_black_bg(subject_id=1, t1_file='T1.nii', t2_file='T2_FLAIR.nii')

# initialization, either by random initialization or k-means
cluster_means, cluster_covariance_matrices, alpha_k = initialize_parameters(tissue_data)

print("cluster_means: ", cluster_means) # cluster_covariance_matrices, alpha_k
print("cluster_covariance_matrices", cluster_covariance_matrices)
# loop over all iterations
    # loop over all k-clusters
        # compute expectation step:
            # -
            # -

        # compute the log-likelihood stopping condition

        # compute maximization step:
            # - 
            # - 



cluster_means:  [array([ 45.9065355 , 210.06372947, 135.37097469]), array([144.90642988, 188.91928569,  58.06819677])]
cluster_covariance_matrices [[array(589.82597431), array(319.20473566), array(490.28011805)], [array(247.83082037), array(441.34737451), array(701.74609958)]]
